In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# 1. 훈련 데이터 전처리하기

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [3]:
char_set = list(set(sentence))
char_dic = {c : idx for idx, c in enumerate(char_set)}
print(char_dic)

{'r': 0, ' ': 1, 'c': 2, 'b': 3, 't': 4, 'l': 5, 'f': 6, 'g': 7, ',': 8, 'k': 9, 'd': 10, 'a': 11, 'p': 12, 'w': 13, 'n': 14, 'e': 15, 'i': 16, 'y': 17, 's': 18, "'": 19, 'u': 20, '.': 21, 'h': 22, 'm': 23, 'o': 24}


In [6]:
dic_size = len(char_dic)
print("문자 집합의 크기 :", dic_size)

문자 집합의 크기 : 25


In [7]:
#하이퍼파라미터 설정
hidden_size = dic_size
sequence_length = 10 # 앞서 만든 샘플을 10개 단위로 끊어서 샘플을 만들 예정
learning_rate = 0.1

In [8]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i : i+sequence_length]
    y_str = sentence[i+1: i+1+sequence_length]
    print(i, x_str, "->", y_str)
    
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [9]:
print(sentence)

if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.


In [10]:
print(x_data[0])
print(y_data[0])

[16, 6, 1, 17, 24, 20, 1, 13, 11, 14]
[6, 1, 17, 24, 20, 1, 13, 11, 14, 4]


In [12]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print(X.shape)
print(Y.shape)

torch.Size([170, 10, 25])
torch.Size([170, 10])


# 2. 모델 구현하기

In [16]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)
    def forward(self,x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [22]:
net = Net(dic_size, hidden_size, 2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)
nb_epochs = 100

In [23]:
outputs = net(X)
print(outputs.shape)

torch.Size([170, 10, 25])


In [24]:
print(outputs.view(-1, dic_size).shape)

torch.Size([1700, 25])


In [25]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [26]:
for epoch in range(nb_epochs + 1):
    outputs = net(X)
    loss = criterion(outputs.view(-1,dic_size), Y.view(-1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        results = outputs.argmax(dim = 2)
        predict_str=""
        for j, result in enumerate(results):
            if j == 0:
                predict_str += ''.join([char_set[t] for t in result])
            else:
                predict_str += char_set[result[-1]]
        print("predict :",predict_str, "loss :", loss)

predict : bccbybbbbbbcbybbcbbbcbbbcbbbbbbbbccbcbbbccbbcbbbbcbybobccbbbcbbbbbbybbbcbbcbbbcbbbbccbbbbbbcbccbbbcbbbcbbbcbbbbbbbcbcbbbccbbbcbbbcbccbbbcbbbbbccbbbbccbbbbbbcbbbbbbcbbbbcbbcbcccbcb loss : tensor(3.2641, grad_fn=<NllLossBackward>)
predict : o tos toe  toslos totothwoe tot totos  totloo ostto  toet toscorlec  to  odos tod totot woteoer totkt tos to  w tot toe ec totco woecetotlo   tot toettot toemto  eoec  to toe tot  loss : tensor(2.2785, grad_fn=<NllLossBackward>)
predict : pntonmwont to coild anth,p, dod't aodm ao peodle to ether to chdlect tool dod dod't ansign them tosk  and dork, ant tothem to ch them to conl ao  themsnd ess ts ihsit  on themshtp loss : tensor(1.2325, grad_fn=<NllLossBackward>)
predict : mntof want to cuild anship, don't arum up people together to collect wood and don't assign them tosks and dork, but rather toach them to cong for the endless immensity of the eesh loss : tensor(0.6644, grad_fn=<NllLossBackward>)
predict : m  ou want to build a ship, don't 